In [2]:
!pip install pyspark pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=3d1478b57a065325252f19b4da9953b062602fd0d8f02d0580c0e6bdfb936fc7
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [14]:
import pyspark
import pandas as pd
import requests
from datetime import datetime, timedelta, date
from pyspark.sql import SparkSession
from pyspark.context import SparkContext as sc
import os
import numpy as np

spark = SparkSession.builder.getOrCreate()

In [4]:
STOCK_SYMBOLS = ['TSLA', 'NVDA', 'MSFT', 'IBM']
# STOCK_SYMBOLS = ['IBM']
csv_data_root = '/content/drive/MyDrive/isdl_data'

In [5]:
def fetch_api_data(stock_symbol):
  request_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={stock_symbol}&interval=5min&outputsize=full&apikey=4DTF3RRYQJERNX8Y'
        # f'&apikey=demo'
  stock_response_obj = requests.get(request_url)
  raw_data = stock_response_obj.json()['Time Series (5min)']
  pandas_df = pd.DataFrame(raw_data).transpose()
  pandas_df = pandas_df.reset_index().rename(columns={
      "index":"Date", '1. open': 'Open', '2. high': 'High','3. low': 'Low', '4. close': 'Close', '5. volume': 'Volume'
      })
  cleaned_pandas_df = clean_raw_df(pandas_df)
  return cleaned_pandas_df


In [6]:
def clean_raw_df(pandas_df):
  pandas_df = pandas_df.astype({
      'Open': float, 'High': float, 'Low': float, 'Close': float, 'Volume': int
  })
  pandas_df['Date'] = pd.to_datetime(pandas_df['Date'])
  return pandas_df

In [7]:
def read_or_create_stock_file(stock_symbol):
  # Open file for given stock symbol
  file_name = f'{csv_data_root}/{stock_symbol}_cleaned.csv'
  if os.path.isfile(file_name):
    existing_stock_df = pd.read_csv(file_name)
    existing_stock_df.sort_values(by='Date', ascending=True, inplace=True)
    # Check last date of entry in stock data
    # If last entry is of today, return it
    last_date_in_existing_data = pd.to_datetime(existing_stock_df['Date'].iloc[-1]).date()
    if last_date_in_existing_data == date.today() - timedelta(days=1):
      return existing_stock_df

    # else, add new data to existing data
    else:
      cleaned_new_df = fetch_api_data(stock_symbol)
      cleaned_new_df = cleaned_new_df[pd.to_datetime(cleaned_new_df['Date']).dt.date > last_date_in_existing_data]
      existing_stock_df.set_index('Date')
      cleaned_new_df.set_index('Date')
      updated_stock_df = pd.concat([existing_stock_df, cleaned_new_df])
      updated_stock_df.reset_index().rename({"index": "Date"})
      updated_stock_df.to_csv(f'{csv_data_root}/{stock_symbol}_cleaned.csv', index=False)
      return updated_stock_df

  # If no existing file, fetch data and save into new file
  else:
    cleaned_new_df = fetch_api_data(stock_symbol)
    cleaned_new_df.to_csv(f'{csv_data_root}/{stock_symbol}_cleaned.csv', index=False)
    return cleaned_new_df

In [24]:
def calculate_ema(series, period):
  ema = pd.Series(np.nan, index=np.arange(series.shape[0]))
  ema[period-1] = (series[:period].sum())/period
  multiplier = (2/(period + 1))

  for i in range(period, series.shape[0]):
    ema.iloc[i] = ((series.iloc[i] - ema.iloc[i-1]) * multiplier) + ema.iloc[i-1]
  return ema

In [34]:
def calculate_macd(df, fast_period=12, slow_period=26, signal_period=9):
  macd_line = calculate_ema(df['Close'], fast_period) - calculate_ema(df['Close'], slow_period)
  signal_line = pd.Series(np.nan, index=np.arange(df.shape[0]))
  signal_line[slow_period-1:] = calculate_ema(macd_line[slow_period-1:], signal_period)
  macd_histogram = macd_line - signal_line
  return macd_line, signal_line, macd_histogram

In [35]:
# main running tab

dict_of_spark_dfs = {}
for stock in STOCK_SYMBOLS:
  stock_data_df = read_or_create_stock_file(stock)
  macd_line, signal_line, macd_histogram = calculate_macd(stock_data_df, 12, 26, 9)
  stock_data_df['MACD_histogram'] = macd_histogram
  stock_data_df['MACD_line'] = macd_line
  stock_data_df['MACD_signal'] = signal_line
  dict_of_spark_dfs[stock] = stock_data_df


In [36]:
dict_of_spark_dfs['IBM']

,Date,Open,High,Low,Close,Volume,MACD_histogram,MACD_line,MACD_signal
0,2024-06-24 04:00:00,172.05,173.59,172.05,172.20,10,NaN,NaN,NaN
1,2024-06-24 04:10:00,172.46,172.90,172.46,172.90,243,NaN,NaN,NaN
2,2024-06-24 04:15:00,173.23,173.61,173.23,173.50,1335,NaN,NaN,NaN
3,2024-06-24 04:30:00,173.48,173.49,173.21,173.49,16,NaN,NaN,NaN
4,2024-06-24 04:35:00,173.22,173.59,173.20,173.50,1098,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3824,2024-07-26 19:30:00,192.00,192.00,191.80,191.80,92,-0.027679,0.023580,0.051259
3825,2024-07-26 19:35:00,191.80,191.80,191.80,191.80,25,-0.044540,-0.004416,0.040124
3826,2024-07-26 19:45:00,192.00,192.00,192.00,192.00,1,-0.040376,-0.010346,0.030030
3827,2024-07-26 19:50:00,192.00,192.00,192.00,192.00,15,-0.035923,-0.014874,0.021049
